In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
import sklearn.model_selection as ms
import xgboost as xgb
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols

In [2]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [4]:
df_test.head()
df_test = df_test.drop(['Id'], axis=1)

In [28]:
df_train_sample = df_train.sample(n=100000)
# df_train_sample = df_train # only uncomment this line if you want to train on the whole dataset

In [29]:
%%capture
#store gap values
Y_train = df_train_sample.gap.values
# #row where testing examples start
# test_idx = df_train_sample.shape[0]
#delete 'Id' column
#delete 'gap' column

df_train_sample = df_train_sample.drop(['gap'], axis=1)
# code from docs about fingerprinting

In [7]:
def rmse(predictions, actual):
    return (np.sum((predictions - actual)**2) / len(predictions))**.5

In [30]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)

def convert_fp(smile):
    m = Chem.MolFromSmiles(smile)
    # http://www.rdkit.org/docs/GettingStartedInPython.html#fingerprinting-and-molecular-similarity
    return AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=1024)

getBitVector = lambda x: x.ToBitString()
getBitVector = np.vectorize(getBitVector)

fingerprints = getBitVector(pd.DataFrame(df_train_sample['smiles'].apply(convert_fp))['smiles'].values)
df = pd.DataFrame(fingerprints, columns = ['fp'])
splitted = df['fp'].apply(lambda x: pd.Series(list((x))))
splitted.index = df_train_sample.index
df_train_sample = pd.concat([df_train_sample, splitted], axis = 1)
df_train_sample.fillna(0, inplace = True)

In [31]:
df_train_sample = df_train_sample.drop(['smiles'], axis=1)

# split training set into validation set to check rmse
X_train, X_validate, y_train, y_validate = ms.train_test_split(df_train_sample, Y_train, test_size=0.2)

In [10]:
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_pred = LR.predict(X_validate)

In [11]:
RF = RandomForestRegressor()
RF.fit(X_train, y_train)
RF_pred = RF.predict(X_validate)

In [12]:
MLP = MLPRegressor(hidden_layer_sizes=(700))
MLP.fit(X_train, y_train)
MLP_pred = MLP.predict(X_validate)

In [13]:
clf = BayesianRidge(compute_score=True)
clf.fit(X_train, y_train)
ridge_pred = clf.predict(X_validate)

In [112]:
# ## Regular Gradient Boosted Model -- replaced in favor of the more advanced XGBoost models
# GB_params = {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2,
#           'learning_rate': 0.1, 'loss': 'ls', 'max_features': 'sqrt'}
# GB = GradientBoostingRegressor(**GB_params)
# GB.fit(X_train, y_train)
# GB_pred = GB.predict(X_validate)

In [32]:
# XGB doesn't accept pandas dataframes, so we convert to numpy arrays
X_train_BST = X_train.as_matrix()
X_validate_BST = X_validate.as_matrix()

def find_best_param(param_list, param, min_val, max_val, skip):
    # function to find the optimal parameter value between min_val and max_val for XGB model
    # param is the parameter to tune
    results = []
    for tune in np.arange(min_val, max_val + 1, skip):
        param_list[param] = tune
        BST = xgb.XGBRegressor(**param_list)
        BST.fit(X_train_BST, y_train)
        BST_pred = BST.predict(X_validate_BST)
        RMSE = rmse(BST_pred, y_validate)
        results.append((tune, RMSE))
        print ("param: ", tune)
        print ("rmse: ", rmse(BST_pred, y_validate))
    smallest = nsmallest(10, results, key = lambda x : x[1])
    print (smallest)
    return smallest[0]

In [75]:
########## HYPERPARAMETER TUNING
# this will take forever unless the number of samples is small (like 1000)
BST_params = {'booster': 'dart', 'learning_rate': 0.1}
param = 'n_estimators'
best = find_best_param(BST_params, param, 1000, 1400, 200)
print (best)
# we did up to 1400, and 1400 was the best... but for the sake of speed let's use 1000

('param: ', 1000)
('rmse: ', 0.17970054114506895)
('param: ', 1200)
('rmse: ', 0.17908713955374544)
('param: ', 1400)
('rmse: ', 0.178436774695998)
[(1400, 0.178436774695998), (1200, 0.17908713955374544), (1000, 0.17970054114506895)]


(1400, 0.178436774695998)

In [81]:
########## HYPERPARAMETER TUNING
param = 'max_depth'
best = find_best_param(BST_params, param, 3, 10, 1)
print (best)
# we tested 3-10, and 5 was best

('param: ', 3)
('rmse: ', 0.1832587202431702)
('param: ', 4)
('rmse: ', 0.1851966430312995)
('param: ', 5)
('rmse: ', 0.18236172756277233)
('param: ', 6)
('rmse: ', 0.1909267809707192)
('param: ', 7)
('rmse: ', 0.19196166503545173)
('param: ', 8)
('rmse: ', 0.1994797497396973)
('param: ', 9)
('rmse: ', 0.1948279825716672)
('param: ', 10)
('rmse: ', 0.19832220767897982)
[(5, 0.18236172756277233), (3, 0.1832587202431702), (4, 0.1851966430312995), (6, 0.1909267809707192), (7, 0.19196166503545173), (9, 0.1948279825716672), (10, 0.19832220767897982), (8, 0.1994797497396973)]


In [139]:
########## HYPERPARAMETER TUNING
param = 'min_child_weight'
best = find_best_param(BST_params, param, 1, 10, 1)
print (best)
# tested 1-10, 9 was the best

('param: ', 1)
('rmse: ', 0.1778751997787211)
('param: ', 2)
('rmse: ', 0.17865408471851757)
('param: ', 3)
('rmse: ', 0.17381468739165046)
('param: ', 4)
('rmse: ', 0.1800128242478271)
('param: ', 5)
('rmse: ', 0.17481356800098266)
('param: ', 6)
('rmse: ', 0.17215753887464105)
('param: ', 7)
('rmse: ', 0.17008854164047796)
('param: ', 8)
('rmse: ', 0.17682972952321385)
('param: ', 9)
('rmse: ', 0.16970147678191272)
('param: ', 10)
('rmse: ', 0.17118508758238754)
[(9, 0.16970147678191272), (7, 0.17008854164047796), (10, 0.17118508758238754), (6, 0.17215753887464105), (3, 0.17381468739165046), (5, 0.17481356800098266), (8, 0.17682972952321385), (1, 0.1778751997787211), (2, 0.17865408471851757), (4, 0.1800128242478271)]


In [33]:
# sets BST_params in accordance with the best hyperparameters as previously calculated
BST_params = {'booster': 'dart', 'learning_rate': 0.1, 'n_estimators': 1000, 'min_child_weight': 9, 'max_depth': 5}

# creates XGB model
BST = xgb.XGBRegressor(**BST_params)
BST.fit(X_train_BST, y_train)
BST_pred = BST.predict(X_validate_BST)

In [37]:
# if we want to save BST model to a file so we don't have to retrain later
import pickle
filename = "bst_model3"
pickle.dump(BST, open(filename, 'wb'))

In [36]:
# print(rmse(LR_pred, y_validate))
# print(rmse(RF_pred, y_validate))
# print(rmse(MLP_pred, y_validate))
# print(rmse(ridge_pred, y_validate))
# print(rmse(GB_pred, y_validate))
print(rmse(BST_pred, y_validate))

0.08321329209794556


In [38]:
# creates prediction vector
BST_pred = np.zeros((len(df_test)))
for i in np.arange(0, len(df_test) + 1, 10000):
    fingerprints = getBitVector(pd.DataFrame(df_test[i:i + 10000]['smiles'].apply(convert_fp))['smiles'].values)
    df = pd.DataFrame(fingerprints, columns = ['fp'])
    splitted = df['fp'].apply(lambda x: pd.Series(list((x))))
    splitted.index = df_test[i:i + len(df)].index
    df_test_step = pd.concat([df_test[i:i + len(df)], splitted], axis = 1)
    df_test_step.fillna(0, inplace = True)
    df_test_step = df_test_step.drop(['smiles'], axis=1)
    BST_pred_step = BST.predict(df_test_step.as_matrix())
    BST_pred[i:i + len(BST_pred_step)] = BST_pred_step
    print (i + 10000)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000


In [39]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [40]:
# write_to_file("sample1.csv", LR_pred)
write_to_file("pred.csv", BST_pred)